# Homework 2

In [3]:
%config IPCompleter.use_jedi = False
import numpy as np
np.set_printoptions(precision=4)
from scipy import stats
import matplotlib.pyplot as plt
import cvxpy as cp
import pandas as pd

from pathlib import Path
fig_path = str(Path().absolute())+'/figures/hw2/'
print(fig_path)

/home/zpyang/grad_courses/2021_spring/ece595_ml/figures/hw2/


## Exercise 1

In [27]:
data_path = str(Path().absolute())+'/hw2/data/'

male_train_data = pd.read_csv(data_path+'male_train_data.csv')
female_train_data = pd.read_csv(data_path+'female_train_data.csv')
m_bmi = male_train_data['male_bmi']*0.1
m_stature = male_train_data['male_stature_mm']*0.001
f_bmi = female_train_data['female_bmi']*0.1
f_stature = female_train_data['female_stature_mm']*0.001

print('Female BMI: '+str(f_bmi.head(10).values))
print('Female Stature(m): '+str(f_stature.head(10).values))
print('Male BMI: '+str(m_bmi.head(10).values))
print('Male Stature(m): '+str(m_stature.head(10).values))

Female BMI: [2.82 2.22 2.71 2.81 2.55 2.3  3.56 3.11 2.46 4.3 ]
Female Stature(m): [1.563 1.716 1.484 1.651 1.548 1.665 1.564 1.676 1.69  1.704]
Male BMI: [3.   2.56 2.42 2.74 2.59 2.53 2.27 2.54 3.41 3.34]
Male Stature(m): [1.679 1.586 1.773 1.816 1.809 1.662 1.829 1.686 1.761 1.797]


array([2.82, 2.22, 2.71, 2.81, 2.55, 2.3 , 3.56, 3.11, 2.46, 4.3 ])